In [631]:
import pandas as pd
import rdflib
import numpy as np
from rdflib import Graph, Namespace, URIRef, BNode, Literal, RDF, util
from rdflib.namespace import NamespaceManager
import datetime
pd.options.display.max_colwidth = 144

In [632]:
time = str(datetime.datetime.now().strftime('%Y_%m_%d'))

In [633]:
def merge_columns(sheet, mapping, origin):
    ## noch ein bisschen sauberer wäre gut, zeilenumbruch nach beschreibung vllt und personennamen in richtiger schreibweise
    ## bei result = texts müssten Autoren noch mit den Labels der Personen, auf die verwiesen wird, versehen werden
    
    df = sheet.tdf
    
    if origin == "zeitraum":
        col1 = mapping["zeitraum1"]
        col2 = mapping["zeitraum2"]
        col3 = mapping["zeitraum"]
        df.loc[df[col1].notna() & df[col2].notna(), col3] = "timespan_" + df[mapping["ID"]] + "_" + df[col1].apply(str).str[:4] + "_" + df[col2].apply(str).str[:4]
        df.loc[df[col1].notna() & df[col2].isna(), col3] = "timespan_" + df[mapping["ID"]] + "_" + df[col1].apply(str).str[:4]
        df.loc[df[col1].isna() & df[col2].notna(), col3] = "timespan_" + df[mapping["ID"]] + "_" + df[col2].apply(str).str[:4]
    
    if origin == "season":
        col1 = mapping["season1"]
        col2 = mapping["season2"]
        col3 = mapping["season"]
        df.loc[df[col1].notna() & df[col2].notna(), col3] = "season_" + df[mapping["ID"]] + "_" + df[col1].apply(str).str[:4] + "_" + df[col2].apply(str).str[:4]
        df.loc[df[col1].notna() & df[col2].isna(), col3] = "season_" + df[mapping["ID"]] + "_" + df[col1].apply(str).str[:4]
        df.loc[df[col1].isna() & df[col2].notna(), col3] = "season_" + df[mapping["ID"]] + "_" + df[col2].apply(str).str[:4]
  
    if origin == "description":
        col1 = mapping["description1"]
        col2 = mapping["description2"]
        col3 = mapping["description"]
        df.loc[df[col1].notna() & df[col2].notna(), col3] = df[col1] + " Quelle: " + df[col2]
        df.loc[df[col1].notna() & df[col2].isna(), col3] = df[col1]
        df.loc[df[col1].isna() & df[col2].notna(), col3] = "Quelle: " + df[col2]        
    
    if origin == "texts":
        col1 = mapping["texts1"]
        col2 = mapping["texts2"]
        col3 = mapping["texts"]
        df[col2] = peoplereplace(df[col2])
        df.loc[df[col1].notna() & df[col2].notna(), col3] = "Text(e): " + df[col1] + " Autor(en): " + df[col2]
        df.loc[df[col1].notna() & df[col2].isna(), col3] = "Text(e): " + df[col1]
        df.loc[df[col1].isna() & df[col2].notna(), col3] = "Autor(en): " + df[col2]
        

    if origin == "condition":
        col1 = mapping["condition1"]
        col2 = mapping["condition2"]
        col3 = mapping["condition"]
        df.loc[df[col1].notna() & df[col2].notna(), col3] = "Zustand: " + df[col1] + " Datum Zustandsaufnahme: " + df[col2].apply(str)
        df.loc[df[col1].notna() & df[col2].isna(), col3] = "Zustand: " + df[col1]
        df.loc[df[col1].isna() & df[col2].notna(), col3] = " Datum Zustandsaufnahme: " + df[col2].apply(str)       

    if origin == "institution":
        col1 = mapping["ID"]
        col2 = mapping["institution1"]
        col3 = mapping["institution"]
        df.loc[df[col2].notna(), col3] = df[col1].str.replace("LOC_", "G_")
        
    if origin == "physical": # neue if-Kondition für objekte, die physisch nicht exisitieren
        regexr = "[\s\S]+"
        col1 = mapping["ID"]
        col3 = mapping["physical"]
        
        if sheet.name == "Objekte VIDEOS":
            regexr = "^(vid-[0-9]{3})$"
            df.loc[df[col1].notna() & df[col1].str.match(regexr, na=False), col3] = "phys_" + df[col1]
            
        if sheet.name == "Objekte BILD":
            regexr = "(Negativmappe)|(Mappe \(sonstige\))|(Fotomappe \(Positive\))"
            col2 = "Archivalientyp"
            df.loc[df[col1].notna() & ~df[col2].str.match(regexr, na=False), col3] = "phys_" + df[col1]
        
        if sheet.name == "Objekte TEXT":
            regexr = "(Archivmappe)|(Mappe \(sonstige\))|(Typoskript Sammlung)|(Anderes)|(Plakate)"
            col2 = "Archivalientyp"
            df.loc[df[col1].notna() & ~df[col2].str.match(regexr, na=False), col3] = "phys_" + df[col1]
            
        if sheet.name == "Objekte AUDIO":
            df[col3] = np.nan
        
    if origin == "digital": # neue if-Kondition für objekte, die digital nicht exisitieren
        
 #      col1 = mapping["ID"]
        col3 = mapping["digital"]
        
        df[col3] = np.nan
 #      df.loc[df[col1].notna(), col3] = "digi_" + df[col1]
 #      df[col4] = df[col3]

In [634]:
def sort_reindex(df, index=None):
    df = (df.sort_values(by=[index])
          .reset_index(drop=True) ## resets index to be continouus            
         )
    return df

In [635]:
def refreplace(col):
    ## string converter for URIs
    col = col.str.lower()
    col = col.str.replace("ä", "ae")
    col = col.str.replace("ö","oe")
    col = col.str.replace("ü","ue")
    col = col.str.replace("ß","ss")
    col = col.str.replace("!?,\.", "")
    col = col.str.replace("[^a-zA-Z0-9;]", "_")
    col = col.str.replace("-", "_")
    col = col.str.replace("_{2,}", "_")
    col = col.str.strip("_")
    col = col.str.strip()
    return col

In [636]:
def supersplit(col):
    ## for more convenient str splitting within the columns (expand=false), mainly for splitting all URIs
    if col.any():
        col = col.str.split(";", expand = False)
    return col

In [637]:
def splitreplace(col):
    ## combines refreplace and supersplit in correct order for they are both always used for any columns containing URIs
    col = refreplace(col)
    col = supersplit(col)
    return col

In [638]:
def peoplereplace(df):
    ## temporary string beautifier for persons as literal values
    df = df.str.replace("_"," ")
    df = df.str.replace(",", " ")
    df = df.str.replace(";", "; ")
    return df

In [639]:
def str_to_rdf(column, nsm):
    newlist = []
    for i in column:
        if isinstance(i, list):
            newlistlist = []
            for j in i:
                newlistlist.append(util.from_n3(j, nsm=nsm))
            newlist.append(newlistlist)
        else:
            newlist.append(float("NaN"))
    column = pd.Series(newlist)
    return column

In [640]:
def mapping_to_dict(mapping):
    mapping_list = []
    merger = pd.DataFrame
    titlekey = pd.DataFrame
    for key,value in mapping.items():
        # value.dropna(subset=["NVTO Property"], inplace=True) # droppt alle Spaltennamen Einträge ohne korrespondierende NVTO Property

        value[['Object Class','NVTO Property']] = value[['Object Class','NVTO Property']].apply(supersplit, axis=1) # splittet alle Spalten mit RDF-Properties oder Classes

        value["NVTO Property"] = str_to_rdf(value["NVTO Property"], nsm) # wandelt die Werte der Spalte in rdflib URIRefs um

        value["Object Class"] = str_to_rdf(value["Object Class"], nsm) # wandelt die Werte der Spalte in rdflib URIRefs um


        merge = (value[['Spaltenname in Tabelle','Merge']].dropna(subset=["Merge"])
                                                        .set_index('Merge').T.to_dict('records')) # merge baut dict zum Mergen von 

        titledict = value.set_index('Spaltenname in Tabelle').T.to_dict('list')
        sheetid = value.loc[value['Merge']=="ID", 'Spaltenname in Tabelle'].iloc[0]
        mapping_list.append(Mapping(sheet = key, #baut Klassenobjekte mit den unten genannten Einträgen
                                    df = value, 
                                    title = value["Spaltenname in Tabelle"], 
                                    rdfp = value["NVTO Property"],
                                    original = value["Merge"],
                                    subj = value["Subject"],
                                    objclass = value["Object Class"],
                                    objtype = value["Object Type"],
                                    merge = merge,
                                    titledict = titledict,
                                    sheetid = sheetid
                                    )
                           )
    return mapping_list

In [641]:
context = Namespace("http://lod.iti-germany.de/contexts/")
dc = Namespace("http://purl.org/dc/elements/1.1/")
dcterms = Namespace("http://purl.org/dc/terms/")
edm = Namespace("http://www.europeana.eu/schemas/edm/")
foaf = Namespace("http://xmlns.com/foaf/0.1/")
mtt_a = Namespace("http://lod.iti-germany.de/schema/mtt_a/")
nvt = Namespace("http://lod.iti-germany.de/resource/")
nvto = Namespace("http://lod.iti-germany.de/schema/nvto/")
owl = Namespace("http://www.w3.org/2002/07/owl#")
rdf = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
rdfs = Namespace("http://www.w3.org/2000/01/rdf-schema#")
skos = Namespace("http://www.w3.org/2004/02/skos/core#")
wgs84_pos = Namespace("http://www.w3.org/2003/01/geo/wgs84_pos#")

In [642]:
nsm = NamespaceManager(Graph())
nsm.bind("context", "http://lod.iti-germany.de/contexts/")
nsm.bind("dc", "http://purl.org/dc/elements/1.1/")
nsm.bind("dcterms", "http://purl.org/dc/terms/")
nsm.bind("edm", "http://www.europeana.eu/schemas/edm/")
nsm.bind("foaf", "http://xmlns.com/foaf/0.1/")
nsm.bind("mtt_a", "http://lod.iti-germany.de/schema/mtt_a/")
nsm.bind("nvt", "http://lod.iti-germany.de/resource/")
nsm.bind("nvto", "http://lod.iti-germany.de/schema/nvto/")
nsm.bind("owl", "http://www.w3.org/2002/07/owl#")
nsm.bind("rdf", "http://www.w3.org/1999/02/22-rdf-syntax-ns#")
nsm.bind("rdfs", "http://www.w3.org/2000/01/rdf-schema#")
nsm.bind("skos", "http://www.w3.org/2004/02/skos/core#")
nsm.bind("wgs84_pos", "http://www.w3.org/2003/01/geo/wgs84_pos#")

In [643]:
def bindbind(graph):
    graph.bind("context", "http://lod.iti-germany.de/contexts/")
    graph.bind("dc", "http://purl.org/dc/elements/1.1/")
    graph.bind("dcterms", "http://purl.org/dc/terms/")
    graph.bind("edm", "http://www.europeana.eu/schemas/edm/")
    graph.bind("foaf", "http://xmlns.com/foaf/0.1/")
    graph.bind("mtt_a", "http://lod.iti-germany.de/schema/mtt_a/")
    graph.bind("nvt", "http://lod.iti-germany.de/resource/")
    graph.bind("nvto", "http://lod.iti-germany.de/schema/nvto/")
    graph.bind("owl", "http://www.w3.org/2002/07/owl#")
    graph.bind("rdf", "http://www.w3.org/1999/02/22-rdf-syntax-ns#")
    graph.bind("rdfs", "http://www.w3.org/2000/01/rdf-schema#")
    graph.bind("skos", "http://www.w3.org/2004/02/skos/core#")
    graph.bind("wgs84_pos", "http://www.w3.org/2003/01/geo/wgs84_pos#")
    return graph

In [644]:
nvt_ds = rdflib.Dataset()
nvt_ds = bindbind(nvt_ds)
pr_graph = nvt_ds.graph(identifier="http://lod.iti-germany.de/contexts/productions")
ev_graph = nvt_ds.graph(identifier="http://lod.iti-germany.de/contexts/events")
vid_graph = nvt_ds.graph(identifier="http://lod.iti-germany.de/contexts/videos")
text_graph = nvt_ds.graph(identifier="http://lod.iti-germany.de/contexts/texts")
img_graph = nvt_ds.graph(identifier="http://lod.iti-germany.de/contexts/images")
aud_graph = nvt_ds.graph(identifier="http://lod.iti-germany.de/contexts/audio")
person_graph = nvt_ds.graph(identifier="http://lod.iti-germany.de/contexts/persons")
group_graph = nvt_ds.graph(identifier="http://lod.iti-germany.de/contexts/groups")
loc_graph = nvt_ds.graph(identifier="http://lod.iti-germany.de/contexts/locations")
city_graph = nvt_ds.graph(identifier="http://lod.iti-germany.de/contexts/cities")
country_graph = nvt_ds.graph(identifier="http://lod.iti-germany.de/contexts/countries")
col_graph = nvt_ds.graph(identifier="http://lod.iti-germany.de/contexts/collections")
series_graph = nvt_ds.graph(identifier="http://lod.iti-germany.de/contexts/series")
concept_graph = nvt_ds.graph(identifier="http://lod.iti-germany.de/contexts/concepts")
nvto_graph_list = [pr_graph, ev_graph, vid_graph, text_graph, img_graph, aud_graph, person_graph, group_graph, loc_graph, city_graph, country_graph, col_graph, series_graph, concept_graph]

In [645]:
edm_ds = rdflib.Dataset()
edm_ds = bindbind(edm_ds)
edm_pr_graph = edm_ds.graph(identifier="http://lod.iti-germany.de/contexts/productions")
edm_ev_graph = edm_ds.graph(identifier="http://lod.iti-germany.de/contexts/events")
edm_vid_graph = edm_ds.graph(identifier="http://lod.iti-germany.de/contexts/videos")
edm_text_graph = edm_ds.graph(identifier="http://lod.iti-germany.de/contexts/texts")
edm_img_graph = edm_ds.graph(identifier="http://lod.iti-germany.de/contexts/images")
edm_aud_graph = edm_ds.graph(identifier="http://lod.iti-germany.de/contexts/audio")
edm_person_graph = edm_ds.graph(identifier="http://lod.iti-germany.de/contexts/persons")
edm_group_graph = edm_ds.graph(identifier="http://lod.iti-germany.de/contexts/groups")
edm_loc_graph = edm_ds.graph(identifier="http://lod.iti-germany.de/contexts/locations")
edm_city_graph = edm_ds.graph(identifier="http://lod.iti-germany.de/contexts/cities")
edm_country_graph = edm_ds.graph(identifier="http://lod.iti-germany.de/contexts/countries")
edm_col_graph = edm_ds.graph(identifier="http://lod.iti-germany.de/contexts/collections")
edm_series_graph = edm_ds.graph(identifier="http://lod.iti-germany.de/contexts/series")
edm_concept_graph = edm_ds.graph(identifier="http://lod.iti-germany.de/contexts/concepts")
edm_graph_list = [edm_pr_graph, edm_ev_graph, edm_vid_graph, edm_text_graph, edm_img_graph, edm_aud_graph, edm_person_graph, edm_group_graph, edm_loc_graph, edm_city_graph, edm_country_graph, edm_col_graph, edm_series_graph, edm_concept_graph]

In [646]:
nvt_master = pd.read_excel('./NVT_Metadatentabelle_MASTER_0609.xlsm', sheet_name=None)
#nvt_master_old = pd.read_excel('./NVT_Metadatentabelle_MASTER_0609test.xlsm', sheet_name=None)

In [647]:
#master_val = [i for i in nvt_master.values()]
#master_val_old = [i for i in nvt_master_old.values()]
#
#
#for idx, i in enumerate(master_val):
#    if any(i != master_val_old[idx]) == True:
#        print("It is true")
#    if i.equals(master_val_old[idx]):
#        print("It is not true")
#    if i.columns.equals(master_val_old[idx].columns):
#        print("heyo")
#    print(master_val_old[idx].columns, "\n", i.columns)

In [648]:
nvto_columns_mapped = pd.read_excel('./Mapping_Meta_Py.xlsx', sheet_name=None)
edm_columns_mapped = pd.read_excel('./Mapping_Meta_Edm.xlsx', sheet_name=None)

In [649]:
class Sheet():
## name: Name des Reiters
## head: Indikator, wo im df Tabellenüberschriften zu finden sind (0 = Inhalte Zeile 1, 1 = Inhalte Zeile 2, x = Index unverändert)
## df: DataFrame pro Sheet
## tdf: Kopie von df, mit über head definierter Indexanpassung
## tdf_empty_transposed: DataFrame, der nur aus Überschriften (ohne Spaltenachsennamen) besteht, zur Konvertierung nach Excel, inklusive Transponierung
    def __init__(self, name, head, df, graph):
        self.name = name
        self.head = head
        self.df = df
        self.graph = graph
        self.tdf = df.copy()
        self.column_names = None
        self.tdf_empty_transposed = None
        
        if self.head != "x":
            self.tdf.columns = self.tdf.iloc[self.head]
            self.tdf.columns.name = None
        else:
            pass
        if isinstance(self.tdf, pd.DataFrame):
            self.column_names = self.tdf.columns.tolist()
            self.tdf_empty_transposed = self.tdf[0:0].transpose(copy=True)

In [650]:
## Zur Erzeungung von Sheet Instanzen mit Name und Spaltenkopfreferenz als Liste und mit dict Referenz
## column_headers: Liste von 'echten' Spaltenüberschriften, 0, 1, für erste, zweite Zeile, x=Standard
## col_ref dict um nicht mit Zahlen, sondern Namen zu arbeiten
## sheet_dict manuell, falls Verschiebungen in Tabelle passieren
old_sheet_list = []
nvto_sheet_list = []
edm_sheet_list = []
nvto_col_ref = {}
edm_col_ref = {}
sheet_dict = {"Produktionen":0,
             "Ereignisse":0,
             "Objekte VIDEOS":0,
             "Objekte TEXT":0,
             "Objekte BILD":0,
             "Objekte AUDIO":0,
             "||_Personen":1,
             "||_Gruppen_Ensembles":"x",
             "||_Veranstaltungsort":"x",
             "||_Städte":"x",
             "||_Länder":"x",
             "Sammlungen":"x",
             "Serie":"x"}

for idx, i in enumerate(sheet_dict.keys()):
 #   old_sheet_list.append(Sheet(name = i, head = sheet_dict[i], df = nvt_master_old[i], graph = graph_list[idx]))
    nvto_sheet_list.append(Sheet(name = i, head = sheet_dict[i], df = nvt_master[i], graph = nvto_graph_list[idx]))
    edm_sheet_list.append(Sheet(name = i, head = sheet_dict[i], df = nvt_master[i], graph = edm_graph_list[idx]))
    nvto_col_ref[i] = nvto_sheet_list[idx]
    edm_col_ref[i] = edm_sheet_list[idx]

In [651]:
sheet_list_list = [nvto_sheet_list, edm_sheet_list]
col_ref_ref = [nvto_col_ref, edm_col_ref]

In [652]:
#writer_trigger = False
#for idx, sheet in enumerate(sheet_list):
#    new_sheet = sheet.tdf
#    old_sheet = old_sheet_list[idx].tdf
#    if new_sheet.columns.equals(old_sheet.columns) == True:
#        print("true")
#        pass
#    else:
#        print("it is equals yes it does")
#        print(new_sheet.columns)
#        print(old_sheet.columns)
#        writer_trigger = True
#        break
## könnte alle abweichenden columns in neuer file gelb markieren

In [653]:
class Mapping():
    
    def __init__(self, sheet, df, title, rdfp, original, subj, objclass, objtype, merge, titledict, sheetid):
        self.sheet = sheet
        self.df = df
        self.title = title
        self.rdfp = rdfp
        self.original = original
        self.subj = subj
        self.objclass = objclass
        self.objtype = objtype
        self.merge = merge
        self.titledict = titledict
        self.sheetid = sheetid

In [654]:
edm_mapping_list = mapping_to_dict(edm_columns_mapped)
nvto_mapping_list = mapping_to_dict(nvto_columns_mapped)

In [655]:
all(elem in list(nvto_mapping_list[0].title)  for elem in nvto_sheet_list[0].column_names) ## gegencheck, ist mapping_meta_py ok?

True

In [656]:
def initial_transform(col_ref):
    
    #bei Produktionen sollte am Ende nach 'XX_' gesucht werden, nicht nach PR, und der Typ bei Produktionen wird dann durch die Typenspalte bestimmt
    tdf = col_ref["Produktionen"].tdf
    tdf = (tdf[tdf['Identifier / geeinigter Name'].str.contains('PR', na=False)] ## drops all rows that don't start with 'PR' or are NaN
              .drop(tdf[tdf['Identifier / geeinigter Name'].str.contains('PR_Internationaler_Workshop_zur_Biomechanik_GITIS Moskau_Januar_1993', na=False)].index)
           )
    col_ref["Produktionen"].tdf = tdf

    tdf = col_ref["Ereignisse"].tdf
    tdf = (tdf[tdf['Identifier / geeinigter Name'].str.contains('EV', na=False)] ## drops all rows that don't start with 'PR' or are NaN
              .drop(tdf[tdf['Identifier / geeinigter Name'].str.contains('EV_Internationaler_Workshop_zur_Biomechanik_GITIS Moskau_Januar_1993_001', na=False)].index) ## drops example line
          )
    col_ref["Ereignisse"].tdf = tdf

    tdf = col_ref["Objekte VIDEOS"].tdf
    tdf = tdf[tdf['Projekt ID'].str.contains('vid', na=False)]
    col_ref["Objekte VIDEOS"].tdf = tdf

    tdf = col_ref["Objekte TEXT"].tdf
    tdf = tdf[tdf['Projekt ID'].str.contains('txt', na=False)]
    col_ref["Objekte TEXT"].tdf = tdf

    tdf = col_ref["Objekte BILD"].tdf
    tdf = tdf[tdf['Projekt ID'].str.contains('img', na=False)]
    col_ref["Objekte BILD"].tdf = tdf

    tdf = col_ref["Objekte AUDIO"].tdf
    tdf = tdf[tdf['Projekt ID'].str.contains('aud', na=False)]
    col_ref["Objekte AUDIO"].tdf = tdf

    tdf = col_ref["||_Personen"].tdf
    tdf = tdf.drop(tdf[tdf['Identifier / geeinigte Schreibweise'].str.contains('Vorname Vatersname,Nachname', na=False)].index)
    col_ref["||_Personen"].tdf = tdf

    tdf = col_ref["||_Gruppen_Ensembles"].tdf
    tdf = tdf[tdf['Gruppe Identifier / geeinigte Schreibweise'].str.contains('G_', na=False)]
    col_ref["||_Gruppen_Ensembles"].tdf = tdf

    tdf = col_ref["||_Veranstaltungsort"].tdf
    tdf = tdf[tdf['PROJEKT'].str.contains('LOC_', na=False)]
    col_ref["||_Veranstaltungsort"].tdf = tdf

    tdf = col_ref["Sammlungen"].tdf
    tdf = (tdf[tdf['PROJEKT'].str.contains('COL_', na=False)]
              .drop(tdf[tdf['PROJEKT'].str.contains('COL_Identifier', na=False)].index)
          )
    col_ref["Sammlungen"].tdf = tdf

    tdf = col_ref["Serie"].tdf      
    tdf = (tdf[tdf['PROJEKT'].str.contains('SRS_', na=False)]
               .drop(tdf[tdf['PROJEKT'].str.contains('SRS_Identifier', na=False)].index)
           )
    col_ref["Serie"].tdf = tdf

In [657]:
for col_ref in col_ref_ref:
    initial_transform(col_ref)

In [658]:
def reindex_merge_splitreplace(sheets, mapping):
        
    # sortiert und reindexiert alle sheetssheets
    
    for sdx, sheet in enumerate(sheets):
            
        sheet.tdf = sort_reindex(sheet.tdf, index=mapping[sdx].sheetid)

    # für alle Spalten in allen Sheets wird merge_columns angewandt
    # mapping[sdx].title = alle Spalten
    # mapping[sdx].merge[0] = ID des Sheets, 0, weil Dict seltsam in Liste
    # mapping[sdx].original[cdx] = für die Keywords in merge_columns, könnte man vllt auch mit merge[0] machen

    for sdx, sheet in enumerate(sheets):
        for cdx, column in enumerate(mapping[sdx].title):
            merge_columns(sheet, mapping[sdx].merge[0], mapping[sdx].original[cdx]) # für die zusammengesetzten Spalten für z.B. Spielzeit Beschreibung Texte

    # Schleife muss getrennt von der oberen laufen, weil in der unteren Listen erzeugt werden, die nicht mehr für die oberen verwendbar sind
    # mapping[sdx].title = alle Spalten
    # mapping[sdx].titledict[column][5] verbindet Spalte mit Keyword "Reference"
    # splitreplace wandelt alle Reference Zeilen zu Listen mit Referenzen um
    # len(sheets[sdx].tdf[column].value_counts()) > 0 prüft die 
    for sdx, sheet in enumerate(sheets):
        for cdx, column in enumerate(mapping[sdx].title):
            reference = mapping[sdx].titledict[column][5]
            if reference == "Reference" and len(sheets[sdx].tdf[column].value_counts()) > 0:
                sheets[sdx].tdf[column] = splitreplace(sheets[sdx].tdf[column])
    
    return sheets

In [659]:
nvto_sheet_list = reindex_merge_splitreplace(nvto_sheet_list, nvto_mapping_list)

In [660]:
edm_sheet_list = reindex_merge_splitreplace(edm_sheet_list, edm_mapping_list)

In [661]:
len(sheet_list[10].tdf["geonames URI"].value_counts()) == 0

True

In [662]:
edm_mapping_list[2].titledict

{'Digitalisiert': [nan, nan, nan, nan, nan, nan, 'mtt_a = MTT administrative'],
 'visibility level': [nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  'mtt_a = MTT administrative'],
 'Projekt ID': ['ID',
  nan,
  [rdflib.term.URIRef('http://www.europeana.eu/schemas/edm/ProvidedCHO')],
  [rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type')],
  nan,
  'Reference',
  nan],
 'andere IDs': [nan,
  nan,
  nan,
  [rdflib.term.URIRef('http://purl.org/dc/elements/1.1/identifier')],
  'Identifier',
  'Literal',
  nan],
 'Standort MCB': [nan, nan, nan, nan, nan, nan, 'mtt_a = MTT administrative'],
 'Kommentar Inhalt': [nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  'mtt_a = MTT administrative'],
 'Kommentar Bearbeitung': [nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  'mtt_a = MTT administrative'],
 'Archivalientyp': [nan,
  nan,
  [rdflib.term.URIRef('http://www.w3.org/2004/02/skos/core#Concept')],
  [rdflib.term.URIRef('http://purl.org/dc/elements/1.1/type')],
  'Type',
  'Reference',
  nan],


In [663]:
list(sheet_list[0].tdf.columns) == list(mapping_list[0].title)

False

In [664]:
def sheets_to_graph(sheet_list, mapping_list):

## prod -> sheet.tdf in sheet_list
## mapping -> mapping_list[sdx]
## for sdx, sheet in enumerate(sheet_list):
## for rdx, row in sheet_list[sdx].tdf.iterrows
    
    for sdx, sheet in enumerate(sheet_list):

        tdf = sheet.tdf
        mapping = mapping_list[sdx]
        graph = sheet.graph

        # Iteration durch die Zeilen eines Sheets, row ist eine ganze Zeile 
        for rdx, row in tdf.iterrows():

            # pro Spalte pro Zeile, column ist der Wert einer Spalte pro Zeile
            for cdx, column in enumerate(row): 
                #Setup der Variablen
                rdfsubj = mapping.titledict[tdf.columns[cdx]][1]
                objclass = mapping.titledict[tdf.columns[cdx]][2]
                rdfprop = mapping.titledict[tdf.columns[cdx]][3]
                objtype = mapping.titledict[tdf.columns[cdx]][5]

                # alle rdfproperties sind in Listen gespeichert, wenn nicht Liste dann NaN
                if tdf.columns[cdx] in list(mapping.title) and isinstance(rdfprop, list): 

                    # pro Property der Spalte
                    for pdx, prop in enumerate(rdfprop): 

                        # nur wenn ein anderes Subjekt in Meta_Py angegeben wird, wird idid angepasst
                        if isinstance(rdfsubj, str) and (isinstance(column, str) or isinstance(column, list)):
                            if isinstance(tdf[rdfsubj][rdx], list):
                                idid = tdf[rdfsubj][rdx][0]
                        else:
                            idid = tdf[mapping.sheetid][rdx][0]

                        #str sind nicht als Liste gespeichert, objtype könnte man noch erweitern auf liste von Länderbezeichungen und Datentypen
                        if isinstance(column, str): 
                            if objtype == "de" or objtype == "en":
                                language = objtype
                            else:
                                language = None  
                            graph.add((nvt[idid], prop, Literal(column, lang=language)))

                        if isinstance(column, list): # References sind als Listen gespeichert
                            for rdfobj in column:
                                if idid != rdfobj:
                                    graph.add((nvt[idid], prop, nvt[rdfobj]))
                                if isinstance(objclass, list):
                                    for iobjclass in objclass:
                                        graph.add((nvt[rdfobj], rdf.type, iobjclass))


        #               if isinstance(column, list) and isinstance(objclass, list): # für zusätzliche Klassenzugehörigkeit (Bourgeoisie, Proletariat)
        #                   triple_object = column[0] # dem Wert der Spalte wird eine Klasse zugewiesen
        #                   for iobjclass in objclass:
        #                       graph.add((nvt[triple_object], prop, iobjclass))               

In [665]:
sheets_to_graph(nvto_sheet_list, nvto_mapping_list)

In [666]:
sheets_to_graph(edm_sheet_list, edm_mapping_list)

In [667]:
with open('nvt_ds_' + str(time) + '.trig', 'wb') as f_trig:
    f_trig.write(nvt_ds.serialize(format="trig"))

In [668]:
with open('edm_ds_' + str(time) + '.trig', 'wb') as f_trig:
    f_trig.write(edm_ds.serialize(format="trig"))